In [2]:
# References
# https://medium.com/data-and-beyond/complete-guide-to-building-bert-model-from-sratch-3e6562228891
# https://ai.plainenglish.io/bert-pytorch-implementation-prepare-dataset-part-1-efd259113e5a

import torch
from torch import nn
from pathlib import Path
from tokenizers import Tokenizer
from huggingface_hub import PyTorchModelHubMixin
import os
import torch
import re
import random
import transformers, datasets
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer
import tqdm
from torch.utils.data import Dataset, DataLoader
import itertools
import math
import torch.nn.functional as F
import numpy as np
from torch.optim import Adam
import math
from tqdm import tqdm

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"

In [4]:
!nvidia-smi

Mon Jun 24 21:54:13 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.03              Driver Version: 555.85         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   45C    P0             13W /   80W |       0MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [60]:
#Hyperparameters
n_warmup_steps = 4000
beta_1 = 0.9
beta_2 = 0.98
epsilon = 1e-9
n_segments = 3
block_size = 128
batch_size = 64
embeddings_dims = 512
attn_dropout = 0.1
no_of_heads = 8 #IMP needs to be thoroughly calculated
dropout = 0.1
epochs = 20
max_lr = 2e-5
no_of_encoder_layers = 6 #IMP needs to be thoroughly calculated

In [745]:
#Data

!wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
!unzip -qq cornell_movie_dialogs_corpus.zip
!rm cornell_movie_dialogs_corpus.zip
!mkdir datasets
!mv cornell\ movie-dialogs\ corpus/movie_conversations.txt ./datasets
!mv cornell\ movie-dialogs\ corpus/movie_lines.txt ./datasets





--2024-06-24 00:28:26--  http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


132.236.207.53
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.53|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9916637 (9.5M) [application/zip]
Saving to: ‘cornell_movie_dialogs_corpus.zip’

     cornell_movie_   2%[                    ] 226.00K   163KB/s               ^C
[cornell_movie_dialogs_corpus.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
note:  cornell_movie_dialogs_corpus.zip may be a plain executable, not an archive
unzip:  cannot find zipfile directory in one of cornell_movie_dialogs_corpus.zip or
        cornell_movie_dialogs_corpus.zip.zip, and cannot find cornell_movie_dialogs_corpus.zip.ZIP, period.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


mkdir: cannot create directory ‘datasets’: File exists


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


mv: cannot stat 'cornell movie-dialogs corpus/movie_conversations.txt': No such file or directory
mv: cannot stat 'cornell movie-dialogs corpus/movie_lines.txt': No such file or directory


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
### loading all data into memory
corpus_movie_conv = './datasets/movie_conversations.txt'
corpus_movie_lines = './datasets/movie_lines.txt'
with open(corpus_movie_conv, 'r', encoding='iso-8859-1') as c:
    conv = c.readlines()
with open(corpus_movie_lines, 'r', encoding='iso-8859-1') as l:
    lines = l.readlines()

### splitting text using special lines
lines_dic = {}
for line in lines:
    objects = line.split(" +++$+++ ")
    lines_dic[objects[0]] = objects[-1]

### generate convo  pairs
pairs = []
for con in conv:
    ids = eval(con.split(" +++$+++ ")[-1]) #Evaluates the string as a list now
    for i in range(len(ids)):
        pair = []
        
        if i == len(ids) - 1:
            break
        # print(ids[i])
        first = lines_dic[ids[i]].strip()  
        second = lines_dic[ids[i+1]].strip() 

        pair.append(' '.join(first.split()[:block_size]))
        pair.append(' '.join(second.split()[:block_size]))
        pairs.append(pair)
        # break
    # break
    

In [7]:
# sent = pairs[0][0]
# ids = tokenizer(sent)['input_ids']
# len(ids), len(sent)

In [8]:
len(pairs) #Total pairs-> 221K

##########W Tokenization #################
# WordPiece tokenizer

### save data as txt file
text_data = []
file_count = 0

def clean_text(text):
    return text.encode('utf-8', 'ignore').decode('utf-8')

for sample in tqdm([x[0] for x in pairs]):
    # cleaned_sample = clean_text(sample)
    text_data.append(sample)

    # once we hit the 10K mark, save to file
    # if len(text_data) == 10000:
with open(f'./datasets/text.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))
        # text_data = []
        # file_count += 1

paths = 'datasets/text.txt'
# print(paths)
### training own tokenizer
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=True
)

tokenizer.train( 
    files=paths,
    vocab_size=30_000, 
    min_frequency=5,
    # limit_alphabet=1000, 
    wordpieces_prefix='##',
    special_tokens=['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]']
    )

if not os.path.exists('./bert-it-1'): os.mkdir('./bert-it-1')
tokenizer.save_model('./bert-it-1', 'bert-it')
tokenizer = BertTokenizer.from_pretrained('./bert-it-1/bert-it-vocab.txt', local_files_only=True)

#Setting vocab size
vocab_size = tokenizer.vocab_size

100%|██████████| 221616/221616 [00:00<00:00, 5819860.72it/s]


/home/yuvrajsingh/anaconda3/envs/py311/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1947: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


In [9]:
class BERTDataset(Dataset):
    def __init__(self, data_pair, tokenizer, seq_len=block_size):

        self.tokenizer = tokenizer
        self.seq_len = seq_len
        self.corpus_lines = len(data_pair)
        self.lines = data_pair

    def __len__(self):
        return self.corpus_lines

    
    def __getitem__(self,item):
        
        #Getting NSP sentences
        sent1, sent2, is_next = self.get_nsp(item)
        
        #Getting masked sentences
        sent1_masked , label1 = self.get_masked_sentences(sent1)
        sent2_masked , label2 = self.get_masked_sentences(sent2)
        
        #Adding CLS and SEP tokens
        sent1_masked_cls_and_sep_aded = [self.tokenizer.vocab['[CLS]']]+ sent1_masked + [self.tokenizer.vocab['[SEP]']]
        sent2_masked_cls_and_sep_aded = sent2_masked + [self.tokenizer.vocab['[SEP]']]
        
        label1_padding_added = [self.tokenizer.vocab['[PAD]']] + label1 + [self.tokenizer.vocab['[PAD]']] #because of [1:-1] thing (I removed CLS and SEP token before) and the middle one because of the added [SEP] token
        label2_padding_added = label2 + [self.tokenizer.vocab['[PAD]']]
        
        #Segment ids
        segment_ids = [1 for _ in range(len(sent1_masked_cls_and_sep_aded))] + [2 for _ in range(len(sent2_masked_cls_and_sep_aded))]
        
        # print("senti", sent1_masked_cls_and_sep_aded)
        # print("sent2", sent2_masked_cls_and_sep_aded)
        # print("label1", label1_padding_added)
        # print("label1", label2_padding_added)
        
        
        #Combine the sentences
        combined_sentence = sent1_masked_cls_and_sep_aded + sent2_masked_cls_and_sep_aded
        combined_labels = label1_padding_added + label2_padding_added
        
        if(len(combined_sentence) > self.seq_len): 
            combined_sentence = combined_sentence[:self.seq_len]
            combined_labels = combined_labels[:self.seq_len]
            segment_ids = segment_ids[:self.seq_len]
        elif (len(combined_sentence) < self.seq_len):
            while(len(combined_sentence) < self.seq_len):
                combined_sentence += [self.tokenizer.vocab['[PAD]']]
                segment_ids += [0]
                combined_labels += [0]
                
        values = {
            'bert_input_masked': combined_sentence,
            'bert_input_labels': combined_labels,
            'segment_ids': segment_ids,
            'is_next': is_next
        }
        # print(values)
        # print(len(combined_labels))
        # print(len(combined_sentence))
        assert len(combined_labels) == len(combined_sentence)
        return {key: torch.tensor(value) for key, value in values.items()} #Must be converted into tensor 
    
    def get_nsp(self,index):
            t1, t2 = self.lines[index][0], self.lines[index][1]
            
            prob = random.random()
            if(prob < 0.5):
                return t1, t2, 1
            else:
                return t1, self.lines[random.randrange(len(pairs))][1], 0
        
    # def get_masked_sentences(self,sentence):
        
    #     # tokens = sentence.split()
    #     # remove cls and sep token
    #     tokens = self.tokenizer(sentence)['input_ids'][1:-1]
    #     mask_label = []
    #     output = []

    #     # 15% of the tokens would be replaced
    #     for token in tokens:
    #         prob = random.random()

    #         # remove cls and sep token
    #         # token_id = tokenizer(token)['input_ids'][1:-1]

    #         if prob < 0.15:
    #             prob /= 0.15

    #             # 80% chance change token to mask token
    #             if prob < 0.8:
    #                 # for i in range(len(token_id)):
    #                     output.append(self.tokenizer.vocab['[MASK]'])
    #                     # output_label.append(token)
    #             # 10% chance change token to random token
    #             elif prob < 0.9:
    #                 # for i in range(len(token_id)):
    #                     output.append(random.randrange(len(self.tokenizer.vocab)))
    #                     # output_label.append(token)
    #             # 10% chance change token to current token
    #             else:
    #                 output.append(token)
    #             mask_label.append(token)
                

    #         else:
    #             output.append(token)
    #             # for i in range(len(token)):
    #             mask_label.append(0)

    #     # flattening (cus every list above is in 2D and will raise an error when passed to TextEmbeddings layer)
    #     output = list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in output]))
    #     mask_label = list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in mask_label]))
    #     assert len(output) == len(mask_label)
    #     return output, mask_label
            
    def get_masked_sentences(self, sentence):
        tokens = self.tokenizer(sentence)['input_ids'][1:-1]
        mask_label = []
        output = []

        for token in tokens:
            prob = random.random()

            if prob < 0.15:
                prob /= 0.15

                if prob < 0.8:
                    output.append(self.tokenizer.vocab['[MASK]'])
                elif prob < 0.9:
                    output.append(random.randrange(len(self.tokenizer.vocab)))
                else:
                    output.append(token)
                mask_label.append(token)
            else:
                output.append(token)
                mask_label.append(0)

        assert len(output) == len(mask_label)
        return output, mask_label
   
        
    

In [8]:
[tokenizer.vocab['[SEP]']]

[2]

In [8]:
# random_word("What? It's on the way. She says she's got something for me.")
# tokenizer("What? It's on the way. She says she's got something for me.")['input_ids'][1:-1]
# tokenizer.vocab['[MASK]']
ls = [1,2,3]
ls += [2,3]
ls

[1, 2, 3, 2, 3]

In [10]:
#Creating an instance of the dataset class
dataset = BERTDataset(data_pair=pairs, tokenizer=tokenizer, seq_len=block_size)

import os
#Creating a dataloader
train_loader = DataLoader(
   dataset, batch_size=batch_size, shuffle=True, pin_memory=False, num_workers=os.cpu_count())


In [11]:
#Test
sample_data = next(iter(train_loader))
# print('Batch Size', sample_data['bert_input_masked'].size())
print(sample_data)
# 3 is MASK
# result = dataset[random.randrange(len(dataset))]
# print(result)
# print(tokenizer.convert_ids_to_tokens(result['bert_input_masked']))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tok

{'bert_input_masked': tensor([[  1, 269, 184,  ...,   0,   0,   0],
        [  1, 983,   3,  ...,   0,   0,   0],
        [  1, 430,   3,  ...,   0,   0,   0],
        ...,
        [  1,  17,  17,  ...,   0,   0,   0],
        [  1,  17,  17,  ...,   0,   0,   0],
        [  1, 182, 253,  ...,   0,   0,   0]]), 'bert_input_labels': tensor([[ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0, 17,  ...,  0,  0,  0],
        [ 0,  0, 34,  ...,  0,  0,  0],
        ...,
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0]]), 'segment_ids': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'is_next': tensor([1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
      

In [11]:
tokenizer("[CLS] What? It's on the way. She says she's got something for me. [SEP] . Yeah! [SEP]")

{'input_ids': [1, 1, 182, 34, 162, 11, 58, 192, 150, 417, 17, 266, 904, 266, 11, 58, 263, 404, 202, 185, 17, 2, 17, 368, 5, 2, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
tokenizer.convert_tokens_to_ids('[SEP]')

2

In [488]:
segment_ids = torch.tensor([0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,2,2])
# segment_ids = [1 for _ in range(len(sent1.split()))] + [2 for _ in range(len(sent2.split()))] 
# if(len(sent1.split()) + len(sent2.split()) > 0):
    # segment_ids += [0 for x in range(block_size - (len(sent1.split()) + len(sent2.split())))]
segment = SegmentEmbeddings()
print(segment(segment_ids))

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.3165, -0.1863,  0.1542,  ..., -0.5941, -0.2662, -1.2164],
        [-0.3165, -0.1863,  0.1542,  ..., -0.5941, -0.2662, -1.2164],
        [-0.3165, -0.1863,  0.1542,  ..., -0.5941, -0.2662, -1.2164]],
       grad_fn=<EmbeddingBackward0>)


In [313]:
sent1 = "Hi hellow my self yuvraj sinfj"
sent2 = "hellow britha myslef"
final_sent = '[CLS] ' + sent1 + ' [SEP] ' + sent2 + ' [SEP]'
print(final_sent)
if(len(final_sent.split()) < block_size):
    while(len(final_sent.split()) !=  block_size):
        final_sent = final_sent + ' ' + '[PAD]'

[CLS] Hi hellow my self yuvraj sinfj [SEP] hellow britha myslef [SEP]


In [316]:
tokenizer(final_sent)

{'input_ids': [1, 1, 636, 904, 113, 218, 2191, 6502, 106, 290, 100, 3406, 112, 100, 2, 904, 113, 3415, 3917, 218, 102, 188, 112, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [314]:
final_sent

'[CLS] Hi hellow my self yuvraj sinfj [SEP] hellow britha myslef [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [315]:
len(final_sent.split())

256

In [299]:
len(final_sent.split())

19

In [295]:
[0, len(sentence_t0_list)-1, len(sent1.split()) + 1]

[0, 3, 7]

In [297]:
get_masked_sentences(final_sent, sent1)

3
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
[5, 4, 6]


('[CLS] Hi hellow my self vanessa [MASK] [SEP] hellow britha myslef i dont know wht yiou saying man [SEP]',
 [0, 0, 0, 0, 0, 6821, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
tokenizer.convert_tokens_to_ids

In [171]:
tokenizer.convert_ids_to_tokens()

[23, 17, 19]

In [175]:
sentence.split()

['hellow', 'myself', 'yuvraj', 'singh']

In [90]:
token_pos

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25]

In [66]:
tokenizer("Hellow")

{'input_ids': [1, 904, 113, 2], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [29]:
tokenizer.all_special_ids

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


In [31]:
tokenizer.all_special_ids

[4, 2, 0, 1, 3]

In [34]:
tokenizer.convert_tokens_to_ids('[SEP]')

2

In [67]:
# Text embeddings
class TextEmbeddings(nn.Module):
    def __init__(
        self,
        vocab_size = vocab_size,
        embeddings_dims = embeddings_dims
    ):
        super().__init__()
        self.embeddings_table = nn.Embedding(num_embeddings = vocab_size, embedding_dim=embeddings_dims, device=device, padding_idx=0) #Just a look up table to convert the toekns_ids to some numbers
        # nn.init.normal_(self.embeddings_table.weight.data, mean=0, std=0.02)

    def forward(self, x):
        return self.embeddings_table(x) 

In [68]:
# Segment embeddings
class SegmentEmbeddings(nn.Module):
    def __init__(
        self,
        n_segments = n_segments,
        embeddings_dims = embeddings_dims
    ):
        super().__init__()
        self.seg_embds = nn.Embedding(num_embeddings = n_segments, embedding_dim=embeddings_dims, device=device, padding_idx=0)
    def forward(self, x):
        return self.seg_embds(x)

In [69]:
#Layer Normalization

class LayerNormalization(nn.Module):
    def __init__(
        self,
        embeddings_dims = embeddings_dims
    ):
        super().__init__()

        self.layer_norm = nn.LayerNorm(normalized_shape=embeddings_dims, device=device)

    def forward(self, x):
        return self.layer_norm(x)

In [70]:
#FeedForward Neural Network

class MLPBlock(nn.Module):
    def __init__(
        self,
        dropout = dropout,
        embeddings_size = embeddings_dims,
        # inner_dimensional_states: int = 3072
    ):
        super().__init__()

        self.mlp = nn.Sequential(
            nn.Linear(device=device, in_features=embeddings_size, out_features= 4 * embeddings_size),
            nn.ReLU(),
            nn.Linear(device=device, in_features= 4 * embeddings_size, out_features=embeddings_size),
            nn.Dropout(p = dropout)
        )

    def forward(self, x):
        # mlp_weights_init = self.mlp.apply(weights_init)
        return self.mlp(x)

In [71]:
#Single Attention Head

class AttentionHead(nn.Module):
    def __init__(
        self,
        attn_dropout = attn_dropout,
        embeddings_dims = embeddings_dims,
        no_of_heads = no_of_heads,
    ):
        super().__init__()
        self.head_size = embeddings_dims // no_of_heads
        self.query = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=device, bias=False)
        self.keys = nn.Linear(in_features=embeddings_dims, out_features=self.head_size,device=device, bias=False)
        self.values = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=device,bias=False)
        self.dropout = nn.Dropout(p = attn_dropout)


    def forward(self, x):
        # batch, block_size, embd_dims = x.shape
        k = self.keys(x)
        q = self.query(x)
        v = self.values(x)
        masked_table = torch.tril(torch.ones(block_size, block_size, device=device))
        weights = q @ torch.transpose(k, dim0=-2, dim1=-1) * (k.shape[-1] ** -0.5)
        masked_values = weights.masked_fill(masked_table[: block_size, : block_size] == 0, 1e-9)
        weights_normalized = nn.functional.softmax(masked_values, dim=-1) #Normalize along the embeddings dimension for all the tokens
        # weights_normalized = self.dropout(weights_normalized)
        out = weights_normalized @ v
        out = self.dropout(out)
        return out

In [72]:
# MHA

class MHA(nn.Module):
    def __init__(
        self,
        attn_dropout = attn_dropout,
        embeddings_dims = embeddings_dims,
        no_of_heads = no_of_heads,
    ):
        super().__init__()
        self.heads = nn.ModuleList([AttentionHead(attn_dropout=attn_dropout, embeddings_dims=embeddings_dims, no_of_heads=no_of_heads) for _ in range(no_of_heads)])
        self.dropout = nn.Dropout(p = attn_dropout)
        self.linear = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=device, bias=False) # 12 (no of heads) * (batch_size) 64 = 768 -> gives out the text embeddings

    def forward(self, x):
        concat = torch.cat([head(x) for head in self.heads], dim=-1)
        linear_layer = self.linear(concat)
        out = self.dropout(linear_layer)
        return out

In [73]:
# MHA
import math
class PositionEmbeddings(nn.Module):
    def __init__(
        self,
        embeddings_dims = embeddings_dims,
        block_size = block_size
    ):
        super().__init__()
        
        self.pos_embd = torch.ones((block_size, embeddings_dims), device=device)
        
    def forward(self,x):
        for pos in range(block_size):
            for i in range(0,embeddings_dims // 2):
                self.pos_embd[pos, 2*i] = math.sin(pos/(10000**((2*i)/embeddings_dims)))
                self.pos_embd[pos, 2*i + 1] = math.cos(pos/(10000**((2*i)/embeddings_dims)))
        
        self.pos_embd.unsqueeze(0)
        return self.pos_embd

In [20]:
# Example usage
# embeddings_dims = 512
# block_size = 128
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pos_emb = PositionEmbeddings(embeddings_dims, block_size)
x = torch.zeros((10, block_size, embeddings_dims), device=device)  # Example input
output = pos_emb(x)
print(output.shape)

torch.Size([128, 256])


In [21]:
block_size / 2

64.0

In [74]:
# Decoder Block

class TransformerEncoderBlock(nn.Module):
    def __init__(
        self,
        attn_dropout = attn_dropout,
        embeddings_dims = embeddings_dims,
        no_of_heads = no_of_heads,
        dropout = dropout
    ):
        super().__init__()

        self.mha = MHA(attn_dropout=attn_dropout, embeddings_dims=embeddings_dims, no_of_heads=no_of_heads)
        self.layer_norm1 = LayerNormalization(embeddings_dims=embeddings_dims)
        self.layer_norm2 = LayerNormalization(embeddings_dims=embeddings_dims)
        self.mlp_block = MLPBlock(dropout=dropout, embeddings_size=embeddings_dims)

    def forward(self, x):
        # x = self.mha(x)
        # x = x + self.layer_norm1(x)
        # x = x + self.mlp_block(x)
        # out = self.layer_norm2(x)
        # x = x + self.mha(self.layer_norm1(x))  #Very important step -> Layer Norm on input and then passes it to the subsequent blocks
        # x = x + self.mlp_block(self.layer_norm2(x)) #Very important step
        x = self.layer_norm1(x + self.mha(x))
        x = self.layer_norm2(x + self.mlp_block(x))

        return x

In [75]:
# Encoder Block

class EncoderModel(nn.Module):
    def __init__(
        self,
        attn_dropout = attn_dropout,
        embeddings_dims = embeddings_dims,
        no_of_heads = no_of_heads,
        block_size = block_size,
        dropout = dropout,
        no_of_encoder_layers = no_of_encoder_layers,
        vocab_size = vocab_size,
        n_segments = n_segments
    ):
        super().__init__()

        self.positional_embeddings = PositionEmbeddings(block_size=block_size, embeddings_dims=embeddings_dims)
        self.text_embds = TextEmbeddings(vocab_size=vocab_size, embeddings_dims=embeddings_dims)
        # self.linear_layer = nn.Linear(in_features=embeddings_dims, out_features=vocab_size, device=device, bias=False) # Takes in logits of dimensions- embeds_dims and converts it into dimension of vocab_size (logits in range of vocab_size)
        # self.layer_norm = LayerNormalization(embeddings_dims=embeddings_dims)
        self.encoder_layers = nn.Sequential(*[TransformerEncoderBlock(embeddings_dims=embeddings_dims, attn_dropout=attn_dropout, no_of_heads=no_of_heads, dropout=dropout) for _ in range(no_of_encoder_layers)])
        self.dropout = nn.Dropout(p = dropout)
        self.seg_embds = SegmentEmbeddings(n_segments=n_segments, embeddings_dims=embeddings_dims)
    
    

    def forward(self, x, segment_ids):
        x = self.text_embds(x) + self.seg_embds(segment_ids) + self.positional_embeddings(x) * math.sqrt(embeddings_dims)
        x = self.dropout(x)
        x = self.encoder_layers(x)
        # x = self.layer_norm(x)
        # out = self.linear_layer(x)
        return x

In [76]:
#NSP

class NSP(nn.Module):
    def __init__(
        self,
        embeddings_dims = embeddings_dims,
    ):
        super().__init__()
        # self.encoder_block = EncoderModel(no_of_encoder_layers=no_of_encoder_layers, attn_dropout=attn_dropout, embeddings_dims=embeddings_dims, no_of_heads=no_of_heads, block_size=block_size, dropout=dropout, vocab_size=vocab_size, n_segments=n_segments)
        self.linear_layer = nn.Linear(in_features=embeddings_dims, out_features=2, device=device)
    

    def forward(self, x,  isnext):
        # print(x.shape)
        # print(x[0,:].shape)
        
        # sigmoid = torch.nn.Sigmoid()
        logits = self.linear_layer(x[:,0,:])
        # print(probs)
        # print(torch.argmax(sigmoid(logits),dim=-1))
        # print(isnext.float())
        # print(isnext.view_as(logits).float())
        # print(torch.argmax(logits, dim=0))
        # print(logits)
        loss = nn.functional.cross_entropy(logits, isnext)
        return loss, logits

In [26]:

nsp =MLM()
enc = EncoderModel()
# out = enc(result['bert_input_masked'], result['segment_ids'])
out = nsp(x = enc(sample_data['bert_input_masked'], sample_data['segment_ids']), mask_labels=sample_data['bert_input_labels'])
out
# out.shape

tensor(9.6210, grad_fn=<NllLossBackward0>)

In [45]:
x = enc(sample_data['bert_input_masked'], sample_data['segment_ids'])
# x.transpose(1,2).shape
x.shape

torch.Size([64, 128, 256])

In [27]:
sample_data



{'bert_input_masked': tensor([[    1,   220,    17,  ...,     0,     0,     0],
         [    1, 18278,    34,  ...,     0,     0,     0],
         [    1,  1126,    34,  ...,     0,     0,     0],
         ...,
         [    1,   269,   266,  ...,     0,     0,     0],
         [    1,  2221,    17,  ...,     0,     0,     0],
         [    1,   399,   211,  ...,     0,     0,     0]]),
 'bert_input_labels': tensor([[  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0, 399,   0,  ...,   0,   0,   0]]),
 'segment_ids': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'is_next': tensor([1, 0, 1, 0, 1, 1, 0, 1, 0, 0,

In [77]:
#MLM

class MLM(nn.Module):
    def __init__(
        self,
        embeddings_dims = embeddings_dims,
        vocab_size = vocab_size
    ):
        super().__init__()
        # self.encoder_block = EncoderModel(no_of_encoder_layers=no_of_encoder_layers, attn_dropout=attn_dropout, embeddings_dims=embeddings_dims, no_of_heads=no_of_heads, block_size=block_size, dropout=dropout, vocab_size=vocab_size, n_segments=n_segments)
        self.linear_layer1 = nn.Linear(in_features=embeddings_dims, out_features=vocab_size, device=device)
        # self.linear_layer2 = nn.Linear(in_features=vocab_size, out_features=block_size, device=device)
        
    

    def forward(self, x,  mask_labels):
        # Get the logits from the linear layer
        logits = self.linear_layer1(x)  # logits: (batch_size, seq_len, vocab_size)
        
        # Reshape logits and mask_labels for cross_entropy
        batch_size, seq_len, vocab_size = logits.shape
        logits = logits.view(batch_size * seq_len, vocab_size)        # logits: (batch_size * seq_len, vocab_size)
        mask_labels = mask_labels.view(-1)          # mask_labels: (batch_size * seq_len)
        
        # Calculate the cross-entropy loss
        loss = nn.functional.cross_entropy(logits, mask_labels)
        
        return loss

In [78]:
#BERT

class BERT(nn.Module):
    def __init__(
        self,
        attn_dropout = attn_dropout,
        embeddings_dims = embeddings_dims,
        no_of_heads = no_of_heads,
        block_size = block_size,
        dropout = dropout,
        vocab_size = vocab_size,
        n_segments = n_segments
    ):
        super().__init__()

        self.mlm = MLM(embeddings_dims=embeddings_dims, vocab_size=vocab_size)
        self.nsp = NSP(embeddings_dims=embeddings_dims)
        self.encoder_layer = EncoderModel(attn_dropout=attn_dropout, embeddings_dims=embeddings_dims, no_of_heads=no_of_heads, no_of_encoder_layers=no_of_encoder_layers, block_size=block_size,dropout=dropout,n_segments=n_segments)

    def forward(self, x, segment_ids, labels, isnext):
        x = self.encoder_layer(x, segment_ids)
        mlm_loss = self.mlm(x, labels)
        nsp_loss, logits = self.nsp(x, isnext)
        return mlm_loss, nsp_loss , logits

In [79]:
model = BERT(embeddings_dims=embeddings_dims, vocab_size=vocab_size)
model = model.to(device)

In [80]:
#Printing a summary of the architecture
from torchinfo import summary
sample_data = {key: value.to(device) for key, value in sample_data.items()}
summary(model=model,
        input_data=(sample_data['bert_input_masked'], sample_data['segment_ids'], sample_data['bert_input_labels'], sample_data['is_next']),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])
# model(result['bert_input_masked'],result['segment_ids'])

Layer (type (var_name))                                 Input Shape          Output Shape         Param #              Trainable
BERT (BERT)                                             [64, 128]            --                   --                   True
├─EncoderModel (encoder_layer)                          [64, 128]            [64, 128, 512]       --                   True
│    └─TextEmbeddings (text_embds)                      [64, 128]            [64, 128, 512]       --                   True
│    │    └─Embedding (embeddings_table)                [64, 128]            [64, 128, 512]       10,959,360           True
│    └─SegmentEmbeddings (seg_embds)                    [64, 128]            [64, 128, 512]       --                   True
│    │    └─Embedding (seg_embds)                       [64, 128]            [64, 128, 512]       1,536                True
│    └─PositionEmbeddings (positional_embeddings)       [64, 128]            [128, 512]           --                   --
│    

In [107]:
x = sample_data['bert_input_masked']
x

tensor([[    1,  3047, 19247,  ...,     0,     0,     0],
        [    1,   368,    17,  ...,     0,     0,     0],
        [    1, 17584,   291,  ...,     0,     0,     0],
        ...,
        [    1,   173,    11,  ...,     0,     0,     0],
        [    1,   173,    11,  ...,     0,     0,     0],
        [    1,   368,    17,  ...,     0,     0,     0]])

In [854]:
pos_embd = PositionEmbeddings()
seg_embd = SegmentEmbeddings()
text_embds = TextEmbeddings()
out = text_embds(sample_data['bert_input_masked']) + seg_embd(sample_data['segment_ids']) + pos_embd(sample_data['bert_input_masked'])
out

tensor([[[ 1.0061,  2.7151, -0.1197,  ...,  1.6429, -1.9797,  1.6565],
         [ 1.3595,  1.3807, -0.6527,  ...,  1.6281, -1.4718,  0.4377],
         [ 1.4273,  0.4243, -0.4965,  ...,  1.6281, -1.4717,  0.4377],
         ...,
         [ 0.9948, -0.1016,  0.1837,  ...,  0.9996,  0.0272,  0.9996],
         [ 0.4520, -0.8920, -0.6786,  ...,  0.9996,  0.0273,  0.9996],
         [-0.5064, -0.8623, -0.9944,  ...,  0.9996,  0.0274,  0.9996]],

        [[ 1.0061,  2.7151, -0.1197,  ...,  1.6429, -1.9797,  1.6565],
         [ 0.6228,  1.3948,  1.7603,  ...,  3.9834, -1.2730,  1.7959],
         [ 0.6500,  2.0493, -0.8632,  ...,  1.7049, -3.7139,  1.3620],
         ...,
         [ 0.9948, -0.1016,  0.1837,  ...,  0.9996,  0.0272,  0.9996],
         [ 0.4520, -0.8920, -0.6786,  ...,  0.9996,  0.0273,  0.9996],
         [-0.5064, -0.8623, -0.9944,  ...,  0.9996,  0.0274,  0.9996]],

        [[ 1.0061,  2.7151, -0.1197,  ...,  1.6429, -1.9797,  1.6565],
         [ 1.1296,  3.1447,  0.4807,  ...,  3

In [826]:
pos_embd(sample_data['bert_input_masked']).shape

torch.Size([128, 512])

In [827]:
seg_embd(sample_data['segment_ids']).shape

torch.Size([64, 256, 512])

In [828]:
text_embds(sample_data['bert_input_masked']).shape

torch.Size([64, 256, 512])

In [81]:
class ScheduledOptim():
    '''A simple wrapper class for learning rate scheduling'''

    def __init__(self, optimizer, embeddings_dims, n_warmup_steps):
        self._optimizer = optimizer
        self.n_warmup_steps = n_warmup_steps
        self.n_current_steps = 0
        self.init_lr = np.power(embeddings_dims, -0.5)

    def step_and_update_lr(self):
        "Step with the inner optimizer"
        self._update_learning_rate()
        self._optimizer.step()

    def zero_grad(self):
        "Zero out the gradients by the inner optimizer"
        self._optimizer.zero_grad()

    def _get_lr_scale(self):
        return np.min([
            np.power(self.n_current_steps, -0.5),
            np.power(self.n_warmup_steps, -1.5) * self.n_current_steps])

    def _update_learning_rate(self):
        ''' Learning rate scheduling per step '''

        self.n_current_steps += 1
        lr = self.init_lr * self._get_lr_scale()

        for param_group in self._optimizer.param_groups:
            param_group['lr'] = lr


In [82]:
#Setting up optimizer and lr scheduler

optimizer = torch.optim.Adam(model.parameters(), lr = max_lr, betas=(beta_1,beta_2), eps=epsilon)
lr_scheduler = ScheduledOptim(optimizer=optimizer,embeddings_dims=embeddings_dims,n_warmup_steps=n_warmup_steps)

In [45]:
for data in train_loader:
    print(data)
    break

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

{'bert_input_masked': tensor([[    1,   231,    11,  ...,     0,     0,     0],
        [    1,   541,   202,  ...,     0,     0,     0],
        [    1,   162,    11,  ...,     0,     0,     0],
        ...,
        [    1,   302,    34,  ...,     0,     0,     0],
        [    1, 12513,    17,  ...,     0,     0,     0],
        [    1,   266,   402,  ...,     0,     0,     0]]), 'bert_input_labels': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'segment_ids': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'is_next': tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0,
        1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 

In [43]:
_,_,logits = model(result['bert_input_masked'], result['segment_ids'], result['bert_input_labels'], result['is_next'])
logits

tensor([[ 0.3277,  0.2688],
        [ 0.0486,  0.3428],
        [ 0.0158,  0.2086],
        [ 0.4024,  0.6013],
        [ 0.0945,  0.5034],
        [ 0.5389,  0.5253],
        [ 0.1614,  0.4149],
        [ 0.2898,  0.1618],
        [-0.0174,  0.4979],
        [ 0.4900,  0.0323],
        [ 0.1157,  0.5928],
        [ 0.3247,  0.4949],
        [ 0.3497,  0.3961],
        [ 0.4306,  0.6082],
        [-0.1206,  0.5410],
        [ 0.4971,  0.3105],
        [ 0.6289,  0.3780],
        [ 0.0587,  0.4536],
        [ 0.3644,  0.0355],
        [-0.0520,  0.9022],
        [-0.0686,  0.5447],
        [ 0.2530,  0.5690],
        [ 0.1160,  0.5040],
        [ 0.1742,  0.2256],
        [ 0.2346,  0.3317],
        [-0.0949,  0.4142],
        [ 0.1664, -0.0305],
        [ 0.0898,  0.1764],
        [ 0.2735,  0.5888],
        [ 0.2276,  0.1259],
        [ 0.3556,  0.7751],
        [ 0.1617,  0.3224],
        [ 0.5495,  0.2966],
        [ 0.1566,  0.5112],
        [ 0.3239,  0.4798],
        [ 0.6981,  0

In [44]:
torch.argmax(nn.functional.softmax(logits, dim=-1), dim=-1)

tensor([0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0], device='cuda:0')

In [83]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [84]:
#Training Loop
from sklearn.metrics import accuracy_score
from tqdm import tqdm

model.train()
loss = []
items = []
for epoch in tqdm(range(epochs)):
    loss = []
    items = []
    correct = 0
    total_instances = 0
    accuracy = []
    for i, data in enumerate(train_loader):
        result = {key: value.to(device) for key,value in data.items()}
        mlm_loss, loss_nsp, logits = model(result['bert_input_masked'], result['segment_ids'], result['bert_input_labels'], result['is_next'])
        loss_tot = mlm_loss + loss_nsp
        loss_tot.backward()
        lr_scheduler.zero_grad()
        lr_scheduler.step_and_update_lr()
        loss.append(loss_tot.item())
        y_pred = torch.argmax(nn.functional.softmax(logits, dim=-1), dim=-1)
        acc = accuracy_score(result['is_next'].cpu(), y_pred.cpu())
        # total_instances += 1
        accuracy.append(acc)
        if(i % 100 == 0 or i == len(train_loader) - 1):
            mean_loss = sum(loss) / len(loss)
            mean_accuracy = sum(accuracy) / len(accuracy)
            print("Steps: ",i, "Epoch: ", epoch , "Train loss: ", mean_loss, "NSP Accuracy: ", mean_accuracy)
            
        

  0%|          | 0/20 [00:00<?, ?it/s]

Steps:  0 Epoch:  0 Train loss:  11.644902229309082 NSP Accuracy:  0.5
